In [1]:
#ini-iPython
import os
os.chdir("/home/vinicius/Dropbox/Projeto Mestrado/Codigos/RoboFEI-HT_Debug/AI/Visual_Memory") #Executando na pasta Vision
import sys
sys.path.append("./iPython")
#end-iPython

---- List of execution parameters ----

In [2]:
#des-iPython import argparse

In [3]:
#des-iPython parser = argparse.ArgumentParser(
#des-iPython     description="Visual Memory",
#des-iPython     epilog= "Responsável por rastrear todos os objetos detectáveis pela visão.\\" \
#des-iPython             "Responsible for tracking all objects detectable by vision."
#des-iPython )

#des-iPython parser.add_argument(
#des-iPython     "--robot_numbers", # Full name
#des-iPython     "--rn", # Abbreviation for the name
#des-iPython     type = int, # Type variable
#des-iPython     help = "Número de robôs que serão rastreados.\\" \
#des-iPython            "Number of robots to be tracked." # Description of the variable
#des-iPython )

#des-iPython parser.add_argument(
#des-iPython     "--frequency_of_execution", # Full name
#des-iPython     "--f", # Abbreviation for the name
#des-iPython     type = float, # Type variable
#des-iPython     help = "Frequência de execução do programa, caso seja zero ira rodar sem interrupções.\\" \
#des-iPython            "Frequency of program execution, if zero will run without interruptions." # Description of the variable
#des-iPython )

#des-iPython args = parser.parse_args()

#ini-iPython
class test():
    robot_numbers = None
    frequency_of_execution = None
args = test()
#end-iPython

---- Imports ----

In [4]:
# Libraries to be used.
import sys
sys.path.append("./include")
sys.path.append("./src")

In [5]:
# The standard libraries used in the Visual Memory System.
import numpy as np
import os
from copy import copy

In [6]:
# Used class developed by RoboFEI-HT
from MainFunctions import * # Declaration the main functions.
from Blackboard import * # Class used to access shared memory.
from Robots import * # Class used to track robot.
from Landmark import * # Class used to track landmark.
from Ball import * # Class used to track ball.
from Speeds import * # Classes used to manage robot speed values.

---- Main Code ----

Starting processes

In [7]:
killedProcess() # Recognize external kill

In [8]:
# Default values
os.system('rm ./Data/config.ini') #debug-iPython
parameters = {
    "robot_numbers": 8,
    "relation_measurement_and_robot": 80,
    "frequency_of_execution": 0.2,
}

# Reading "config.ini" values
conf = ConfigIni("Basic", "Settings")
parameters = conf.read(parameters)
if args.robot_numbers != None:
    parameters["robot_numbers"] = args.robot_numbers

if args.frequency_of_execution != None:
    parameters["frequency_of_execution"] = args.frequency_of_execution

if parameters['frequency_of_execution'] != 0:
    parameters['frequency_of_execution'] = 1.0/parameters['frequency_of_execution']

print "Parâmetros:", parameters, "\n" #debug-iPython

# List with velocities for each movement of the robot.
speeds = Speeds( )

mem = Blackboard( ) # Creating shared memory

land = Landmark(speeds) # Creating landmark object

ball = Ball(speeds) # Creating landmark object

# Creating robot objects
observation = []
robots = []
newrobots = []
for __ in xrange(parameters["robot_numbers"]):
    newrobots.append(Robots(speeds, observation))
print "Qnt. robôs:", len(newrobots) #debug-iPython

Parâmetros: {'relation_measurement_and_robot': 80, 'robot_numbers': 8, 'frequency_of_execution': 5.0} 

Qnt. robôs: 8


Main loop

In [9]:
#des-iPython while True:
#ini-tab
try:
    # start timestamp
    start = time.time()
    
    # Reading data about the landmark.
    mem.write_float("VISION_LAND_TAG", 2) #debug-iPython
    taglandmark = mem.read_float("VISION_LAND_TAG")
    print "taglandmark:", taglandmark #debug-iPython
    
    if taglandmark != 0:
        print "Lendo valores de Landmarks" #debug-iPython
        # Read the values sent
        datalandmarks = readDataLandmarks(taglandmark, mem)
        print "Dados landmarks:", datalandmarks #debug-iPython
        speeds.update(land.update(datalandmarks))
        
    else:
        print "Sem valores de Landmarks" #debug-iPython
        land.prediction( )
    
#     datarobots = readDataRobots(mem, parameters["robot_numbers"])
    datarobots = [
        [1, 5, 0, 0],
        [1, 5, 0, 1],
        [2, 5, 0, 2],
        [2, 5, 0, 3],
        [2, 5, 0, 4],
        [1, 5, 0, 5],
        [2, 5, 0, 6],
        [1, 5, 0, 7],
        [2, 5, 0, 8],
        [2, 5, 0, 9],
        [2, 5, 0, 10],
        
        [1, 0, 0, 0],
        [1, 1, 1, 1],
        [1, 2, 2, 2],
        [2, 3, 3, 3],
        [1, 4, 4, 4],
        [2, 5, 5, 5],
        [1, 6, 6, 6],
        [1, 7, 7, 7],
        [1, 8, 8, 8],
        [2, 9, 9, 9],
        [2, 10, 10, 10],
        
        [2, 10, 10, 0],
        [2, 9, 9, 1],
        [2, 8, 8, 2],
        [2, 7, 7, 3],
        [3, 6, 6, 4],
        [3, 5, 5, 5],
        [2, 4, 4, 6],
        [3, 3, 3, 7],
        [3, 2, 2, 8],
        [3, 1, 1, 9],
        [2, 0, 0, 10],
        
        [2, 5, 5, 0],
        [3, 7, 6, 1],
        [3, 9, 7, 2],
        [2, 11, 8, 3],
        [2, 13, 9, 4],
        [3, 15, 10, 5],
        [3, 15, 10, 6],
        [2, 15, 10, 7],
        [3, 15, 10, 8],
        [3, 15, 10, 9],
        [3, 15, 10, 10],
    ]
    
    datarobots.sort(key = lambda a: a[3]); #print datarobots #debug-iPython
    
    if datarobots != []:
        oldtime = datarobots[0][3]
        robotstesting = copy(robots)
        
        obs = 0
        while(obs < len(datarobots)):
            print "Entrou", len(datarobots), "passo:", obs
            if oldtime != datarobots[obs][3]:
                print "Novos dados"
                robotstesting = copy(robots)
                oldtime = datarobots[obs][3]
            
            if robotstesting == []:
                print "Novo Robô"
                newrobots[0].insertObservation(datarobots[obs])
                robots.append(newrobots.pop(0))
                datarobots.remove(datarobots[obs])
                continue
            
            observation = datarobots[obs]
            
            print 'remove todos os elementos de robôs de cores erradas'
            
            sort(robots, reverse = True)
            
            if datarobots[0].weight < float(parameters['relation_measurement_and_robot'])/100:
                if newrobots != []:
                    newrobots[0].insertObservation(datarobots[obs])
                    robots.append(newrobots.pop(0))
                    datarobots.remove(datarobots[obs])
                    obs -= 1
                    continue
            else:
                datarobots[0].insertObservation(datarobots[obs])
                datarobots.pop(0)
                datarobots.remove(datarobots[obs])
                obs -= 1
            
            obs += 1
                    
            
                
        
        
    # stop timestamp
    diff = time.time() - start
    print 'Tempo desejado para espera:', parameters["frequency_of_execution"], 's' #debug-iPython
    print 'Tempo de execução:', diff*1000, 'ms' #debug-iPython
    
    if parameters["frequency_of_execution"] - diff > 0:
        print 'Esperando:', (parameters["frequency_of_execution"] - diff)*1000, 'ms' #debug-iPython
        time.sleep(parameters["frequency_of_execution"] - diff)
        print 'Tempo esgotado' #debug-iPython
    
except KeyboardInterrupt:
    os.system("clear") # Cleaning terminal
    print "Keyboard interrupt detected"
    #des-iPython break
except VisualMemoryException as e:
    #edes-iPython break
    pass
#end-tab -- Fim do loop

taglandmark: 2.0
Lendo valores de Landmarks
Dados landmarks: [2.0, 0.0, 0.0, 0.0]


AttributeError: 'Landmark' object has no attribute 'update'

Finishing processes

In [ ]:
# Saving config values
if parameters['frequency_of_execution'] != 0:
    parameters['frequency_of_execution'] = 1.0/parameters['frequency_of_execution']
conf.finalize(parameters)

In [ ]:
#ini-iPython

# Testes

In [ ]:
robots

In [ ]:
datarobots

In [ ]:
newrobots

In [ ]:
#end-iPython